### kobert_test.ipynb와 다른점
- softmax를 사용하여 기사 제목 및 내용별 긍부정 결과 뿐만 아닌 확률값 도출 가능
- 배치사이즈를 주어서 배치별로 모델 적용을 시켜 훨씬 빠른 결과값 도출 가능

### 라이브러리 로드

In [114]:
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch

from kobert import get_pytorch_kobert_model
from kobert import get_tokenizer

import gluonnlp as nlp

from IPython.display import clear_output
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

### 1. 모델 불러오기

##### 데이터셋 클래스 정의 
- 모델 학습할 때 사용한 데이터셋 클래스 정의, 입력 데이터와 레이블을 적절한 형태로 변환하여 제공하기 위함

In [115]:
# dataset
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([str(i[sent_idx])]) for i in dataset]
        self.labels = [i[label_idx] for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

##### 하이퍼파라미터 및 device 지정
- 모델 학습 시 사용한 하이퍼파라미터 및 장치 설정을 동일하게 지정

In [116]:
# hyperparameter
seed = 0
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [117]:
device

'cuda'

##### 모델 클래스 정의와 인스턴스화
- 학습할 때 사용한 모델 클래스 정의 및 해당 클래스를 사용한 모델 객체 인스턴스화를 통해 모델 구조와 설정 재현

In [118]:
# KoBERT 모델
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size=768, num_classes=2, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size , num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    if self.dr_rate:
        out = self.dropout(pooler)
    else:
        out = pooler
    return self.classifier(out)

In [119]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)  # 모델 클래스를 정의하고 인스턴스화

using cached model. /home/hnu/바탕화면/Lab/.cache/kobert_v1.zip
using cached model. /home/hnu/바탕화면/Lab/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/hnu/바탕화면/Lab/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


##### 저장된 모델 가중치 로드
- 학습된 모델 가중치

In [120]:
# 저장된 모델의 가중치 로드
model.load_state_dict(torch.load("naverShoppingReview_state_dict.pt"))

<All keys matched successfully>

### 2. 모델 적용 예시
- 클래스는 총 2개(부정, 긍정)이며 부정, 긍정에 대한 소프트맥스 값 비교
- 각 클래스에 대한 확률값을 비교해 더 높은 값의 클래스를 예측해줌

In [121]:
# 예측 함수 생성
def predict_proba(predict_sentence):
    predict_sentence = predict_sentence.reshape(-1, 1)
    zeros = np.zeros((len(predict_sentence), 1), dtype=np.int32)

    dataset_another = np.concatenate([predict_sentence, zeros], axis=1)
    test_dataset = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(predict_sentence), num_workers=5, shuffle=False)
    
    model.eval()
    
    all_scores = []
    with torch.no_grad():
        for token_ids, valid_length, segment_ids, label in test_loader:
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length = valid_length.to(device)
            label = label.long().to(device)

            out = model(token_ids, valid_length, segment_ids)
            scores = torch.softmax(out, dim=1)
            all_scores.extend(scores.detach().cpu().numpy())
    
    total_result = list(map(lambda x: '부정' if x[0] > x[1] else '긍정', all_scores))
    positive_score = list(map(lambda x: x[1], all_scores)) # ['부정', '긍정', '부정']
    return total_result, positive_score  # 긍부정 판단반환값, 긍정 클래스에 대한 확률

In [122]:
def concat_df(col_name):
    # 전체 기사 concat
    files = os.listdir(f"{col_name}2/")
    dfs = []

    for file in tqdm(files):
        file_path = os.path.join(f"{col_name}2", file)
        try : 
            df = pd.read_csv(file_path)
        except pd.errors.EmptyDataError :
            continue
        dfs.append(df)

    dfs = pd.concat(dfs, ignore_index=True)
    return dfs

In [123]:
def get_test_batch(dfs, batch_size):
    title_content_values = dfs[['기사제목', '기사내용']].values
    title_content_values_len = len(title_content_values)
    
    test_batch_size = int(np.floor(title_content_values_len / batch_size))
    
    sentence_split = np.array_split(title_content_values, test_batch_size)
    sentence_split_len = len(sentence_split)
    return sentence_split, sentence_split_len

In [124]:
def get_total_result(col_name):
    ## 데이터 합치기
    dfs = concat_df(col_name)

    ## test batch 생성
    sentence_split, sentence_split_len = get_test_batch(dfs, 1024)

    ## 결과 추출
    result_dict = {
        '기사제목_감정분석': [],
        '기사제목_긍정스코어': [],
        '기사내용_감정분석': [],
        '기사내용_긍정스코어': [],
    }

    with tqdm(total=len(sentence_split)) as pbar:
        for idx, batch_sample in enumerate(sentence_split):
            batch_title_sample = batch_sample[:, 0].copy()
            batch_content_sample = batch_sample[:, 1].copy()

            text_score1, float_score1 = predict_proba(batch_title_sample)
            result_dict['기사제목_감정분석'] += text_score1
            result_dict['기사제목_긍정스코어'] += float_score1

            text_score2, float_score2 = predict_proba(batch_content_sample)
            result_dict['기사내용_감정분석'] += text_score2
            result_dict['기사내용_긍정스코어'] += float_score2
            pbar.update(1)

    result = pd.DataFrame(result_dict)
    dfs = pd.concat([dfs, result], axis = 1)

    file = dfs[['기사제목','기사제목_감정분석','기사제목_긍정스코어','기사내용', '기사내용_감정분석','기사내용_긍정스코어','언론사', '년도', '월', 'url']]
    file.to_csv(f"sentiment_analysis_{col_name}.csv", index=False)

In [125]:
get_total_result('항만')
# get_total_result('무역')

  0%|          | 0/7305 [00:00<?, ?it/s]

100%|██████████| 780/780 [28:32<00:00,  2.20s/it]


---

In [131]:
result_df = pd.read_csv('sentiment_analysis_무역.csv')